# Required packages

In [1]:
!pip install mysql-connector-python

In [1]:
from mysql import connector

# Connecting to Mysql

In [3]:
def connectMySQL():
    try :
        conn = connector.Connect(host="localhost",    # your host, usually localhost
                                         user="root",         # your username
                                         port="3306"         # port (3306 default)
                                         )      
        # Creating the cursor
        cur = conn.cursor()
        print("Connected to MySQL")
        return conn,cur
    
    except connector.Error as e:
        print(e)
        return e
        
    

connectMySQL()


Connected to MySQL


(<mysql.connector.connection_cext.CMySQLConnection at 0x2446e464190>,
 <mysql.connector.cursor_cext.CMySQLCursor at 0x2446e464280>)

# Creating a database

In [16]:
def createDb(nameDB):
    conn,cur=connectMySQL()
    try :
        cur.execute(f"Create database {nameDB}")
        cur.execute("commit")
        # Closing the connection to the default database
        conn.close()
        
    except connector.Error as e :
        print(e)

createDb("DVDrentalDB")
    

Connected to MySQL
1007 (HY000): Can't create database 'dvdrentaldb'; database exists


# Connecting to a database

In [4]:
def connectDB(nameDB):
    try :
        conn = connector.Connect(host="localhost",    # your host, usually localhost
                                 user="root",         # your username
                                 port="3306",         # port (3306 default)
                                 db=nameDB
                                         )      
        # Creating the cursor
        cur = conn.cursor()
        print(f"Connected to {nameDB}")
        return conn,cur
    
    except connector.Error as e:
        print(e)
        return e
    
conn,cur=connectDB("DVDrentalDB")

Connected to DVDrentalDB


# Creating tables

In [6]:
def generateTableCreation():
    tableName=input("Enter the name of the Table : ")
    numberColumns=int(input(f"How many columns are there in {tableName} ? "))
    columns={}
    for i in range(numberColumns):
        nameColumn=input(f"Name of the column N°{i+1} : ")
        while nameColumn in columns.keys():
            nameColumn=input(f"Please choose another name for the column N°{i+1} : ")
        typeColumn=input(f"type of the column N°{i+1} : ")
        columns[nameColumn]=typeColumn
    
    primaryKey=input("Choose the primary key : ")
    while primaryKey not in columns.keys():
        primaryKey=input("Choose a primary key from the existing columns : ")

    
    foreignKeys={}
    numberFk=int(input("how many foreign keys are there ? : "))
    while numberFk > len(columns)-1 and numberFk<0:
        numberFk=int(input("Please enter a valid number of foreign keys : "))
    
    for i in range(numberFk):
        foreignKey=input(f"Enter foreign key N°{i+1} : ")
        while (foreignKey not in columns.keys() ) or  (foreignKey in foreignKeys.keys()):
            foreignKey=input(f"Please enter a valid foreign key : ")
        
        referenceTable=input(f"Enter the reference Table name for the {foreignKey} : ")
        referenceName=input(f"Enter the reference name of {foreignKey} in the {referenceTable} table : ")
        foreignKeys[foreignKey]=[referenceTable,referenceName]
    
    print("\n----------------------\n")
        
    print('"""')
    
    print(f"    CREATE TABLE {tableName} (")
    print(f"    {primaryKey} {columns[primaryKey]} PRIMARY KEY ",end="")
    for k,v in columns.items():
        if k==primaryKey:
            continue
        print(f",\n    {k} {v}",end="")
    
    for key in foreignKeys:
        print(f",\n    FOREIGN KEY ({key}) REFERENCES {foreignKeys[key][0]}({foreignKeys[key][1]})")
    
    print("    );")
    print('"""')
            

In [7]:
generateTableCreation()

Enter the name of the Table : Mytab
How many columns are there in Mytab ? 4
Name of the column N°1 : name
type of the column N°1 : varchar(20)
Name of the column N°2 : adress
type of the column N°2 : varchar(20)
Name of the column N°3 : vill
type of the column N°3 : varchar(20)
Name of the column N°4 : id
type of the column N°4 : varchar(20)
Choose the primary key : id
how many foreign keys are there ? : 0

----------------------

"""
    CREATE TABLE Mytab (
    id varchar(20) PRIMARY KEY ,
    name varchar(20),
    adress varchar(20),
    vill varchar(20)    );
"""


In [51]:
def createTableWithSql(cur,conn,sqlQuery):
    try:
        cur.execute(sqlQuery)
        conn.commit()
        
        return "Table created!!"
    except connector.Error as e:
        return e



+ **Country table**

In [9]:
createTableWithSql(cur,conn,"""
    CREATE TABLE country (
    country_id varchar(20) PRIMARY KEY ,
    country char(20),
    last_update datetime    );
"""
)

'Table created!!'

+ **City table**

In [10]:
createTableWithSql(cur,conn,"""
    CREATE TABLE city (
    city_id varchar(20) PRIMARY KEY ,
    city char(20),
    country_id varchar(20),
    last_update datetime,
    FOREIGN KEY (country_id) REFERENCES country(country_id)
    );
"""
)

'Table created!!'

+ **Address table**

In [11]:
createTableWithSql(cur,conn,"""
    CREATE TABLE address (
    address_id varchar(20) PRIMARY KEY ,
    address varchar(100),
    address2 varchar(100),
    district varchar(40),
    city_id varchar(20),
    postal_code numeric(6),
    phone numeric(10),
    last_update datetime,
    FOREIGN KEY (city_id) REFERENCES city(city_id)
    );
"""
)

'Table created!!'

+ **Language table**

In [12]:
createTableWithSql(cur,conn,"""
    CREATE TABLE language (
    language_id varchar(5) PRIMARY KEY ,
    name char(20),
    last_update datetime    );
"""
)

'Table created!!'

+ **Film table**

In [13]:
createTableWithSql(cur,conn,"""
    CREATE TABLE film (
    film_id varchar(20) PRIMARY KEY ,
    title varchar(50),
    description char(200),
    release_year year,
    language_id varchar(5),
    rental_duration datetime,
    rental_rate numeric(2),
    length numeric(5),
    replacement_cost numeric(5,2),
    rating numeric(2),
    last_update datetime,
    special_features char(200),
    full_text varchar(100),
    FOREIGN KEY (language_id) REFERENCES language(language_id)
    );
"""
)

'Table created!!'

+ **Category table**

In [14]:
createTableWithSql(cur,conn,"""
    CREATE TABLE category (
    category_id varchar(20) PRIMARY KEY ,
    name varchar(20),
    last_update datetime    );
"""
)

'Table created!!'

+ **film_category table**

In [15]:
createTableWithSql(cur,conn,"""
    CREATE TABLE film_category (
    film_id varchar(20) PRIMARY KEY ,
    category_id varchar(20),
    last_update datetime,
    FOREIGN KEY (film_id) REFERENCES film(film_id)
,
    FOREIGN KEY (category_id) REFERENCES category(category_id)
    );
"""
)

'Table created!!'

+ **Actor table**

In [16]:
createTableWithSql(cur,conn,"""
    CREATE TABLE actor (
    actor_id varchar(20) PRIMARY KEY ,
    first_name varchar(20),
    last_name varchar(20),
    last_update datetime
    );
"""
)

'Table created!!'

+ **Film_actor table**

In [17]:
createTableWithSql(cur,conn,"""
    CREATE TABLE film_actor (
    actor_id varchar(20) PRIMARY KEY, 
    film_id varchar(20),
    last_update datetime,
    FOREIGN KEY (film_id) REFERENCES film(film_id)
,
    FOREIGN KEY (actor_id) REFERENCES actor(actor_id)
    );
"""
)

'Table created!!'

+ **Store table**

In [53]:
cur.execute(f"DROP TABLE IF EXISTS {store};")
createTableWithSql(cur,conn,"""
    CREATE TABLE store (
    store_id varchar(20) PRIMARY KEY ,
    manager_staff_id varchar(20) ,
    address_id varchar(20),
    last_update datetime,
    FOREIGN KEY (address_id) REFERENCES address(address_id)
    );
"""
)

NameError: name 'store' is not defined

+ **Staff table**

In [19]:
createTableWithSql(cur,conn,"""
    CREATE TABLE staff (
    staff_id varchar(20) PRIMARY KEY ,
    first_name varchar(20),
    last_name varchar(20),
    address_id varchar(20),
    email varchar(20),
    active BOOLEAN,
    username varchar(20),
    password varchar(255),
    last_update datetime,
    FOREIGN KEY (address_id) REFERENCES address(address_id)
    );
"""
)

'Table created!!'

+ **Adding FOREIGN KEYS for Staff and Store tables**

In [31]:
createTableWithSql(cur,conn,"""
    ALTER TABLE store ADD COLUMN manager_id varchar(20)
    ;
"""
)

'Table created!!'

In [8]:
createTableWithSql(cur,conn,"""
    ALTER TABLE store ADD CONSTRAINT fk_store_staff 
    FOREIGN KEY (manager_id) REFERENCES staff(staff_id)
""")

'Table created!!'

mysql.connector.errors.ProgrammingError(1064,
                                        "1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 'FOREIGN KEY (manager_id) REFERENCES staff(staff_id)' at line 1",
                                        '42000')

In [36]:
createTableWithSql(cur,conn,"""
    ALTER TABLE staff ADD COLUMN store_id varchar(20)
    ;
"""
                  )

'Table created!!'

In [12]:
createTableWithSql(cur,conn,"""
    ALTER TABLE staff ADD CONSTRAINT fk_staff_store
    
    FOREIGN KEY (store_id) REFERENCES store(store_id)
   
"""
)

'Table created!!'

mysql.connector.errors.ProgrammingError(1064,
                                        "1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 'FOREIGN KEY (store_id) REFERENCES store(store_id)' at line 1",
                                        '42000')

+ **Customer table**

In [26]:
createTableWithSql(cur,conn,"""
    CREATE TABLE customer (
    customer_id varchar(20) PRIMARY KEY ,
    store_id varchar(20),
    first_name varchar(20),
    last_name varchar(20),
    email varchar(20),
    address_id varchar(20),
    activebool BOOLEAN,
    create_date datetime,
    last_update datetime,
    active varchar(20),
    
    FOREIGN KEY (store_id) REFERENCES store(store_id)
    );
"""
)

'Table created!!'

+ **Inventory table**

In [27]:
createTableWithSql(cur,conn,"""
    CREATE TABLE inventory (
    inventory_id varchar(20) PRIMARY KEY ,
    film_id varchar(20),
    store_id varchar(20),
    last_update datetime,
    
    FOREIGN KEY (film_id) REFERENCES film(film_id),
    FOREIGN KEY (store_id) REFERENCES store(store_id)
    );
"""
)

'Table created!!'

+ **Rental table**

In [28]:
createTableWithSql(cur,conn,"""
    CREATE TABLE rental (
    rental_id varchar(20) PRIMARY KEY ,
    rental_date datetime,
    inventory_id varchar(20),
    customer_id varchar(20),
    return_date datetime,
    staff_id varchar(20),
    last_update datetime,
    
    FOREIGN KEY (inventory_id) REFERENCES inventory(inventory_id),
    FOREIGN KEY (customer_id) REFERENCES customer(customer_id),
    FOREIGN KEY (staff_id) REFERENCES staff(staff_id)
    );
"""
)

'Table created!!'

+ **Payment table**

In [29]:
createTableWithSql(cur,conn,"""
    CREATE TABLE payment (
    payment_id varchar(20) PRIMARY KEY ,
    customer_id varchar(20),
    staff_id varchar(20),
    rental_id varchar(20),
    amount varchar(20),
    payment_date datetime,
    
    FOREIGN KEY (customer_id) REFERENCES customer(customer_id),
    FOREIGN KEY (staff_id) REFERENCES staff(staff_id),
    FOREIGN KEY (rental_id) REFERENCES rental(rental_id)
    );
"""
)

'Table created!!'

mysql.connector.errors.IntegrityError(1451,
                                      '1451 (23000): Cannot delete or update a parent row: a foreign key constraint fails',
                                      '23000')

# Inserting data

In [19]:
!pip install pymysql
import pymysql
import sys

In [5]:
def csv_to_mysql(my_query):

    try:
        conn = pymysql.connect(host="localhost",
                                user="root",
                                port="3306",
                                autocommit=True,
                                local_infile=1)
        print('Connected to DB: {}'.format(host))
        # Create cursor and execute Load SQL
        cur = conn.cursor()
        cur.execute(my_query)
        print('table loaded from csv, mbrouk')
        conn.close()
        
    except Exception as e:
        print('Error: {}'.format(str(e)))
        sys.exit(1)

In [6]:
#Importing pandas
import pandas as pd

#### Inserting data into actor table

In [22]:
actordata = pd.read_csv (r'..\data\actor-1677886980800.csv')   
df = pd.DataFrame(actordata)
print(df)

     actor_id first_name     last_name             last_update
0           1   Penelope       Guiness  2013-05-26 14:47:57.62
1           2       Nick      Wahlberg  2013-05-26 14:47:57.62
2           3         Ed         Chase  2013-05-26 14:47:57.62
3           4   Jennifer         Davis  2013-05-26 14:47:57.62
4           5     Johnny  Lollobrigida  2013-05-26 14:47:57.62
..        ...        ...           ...                     ...
195       196       Bela        Walken  2013-05-26 14:47:57.62
196       197      Reese          West  2013-05-26 14:47:57.62
197       198       Mary        Keitel  2013-05-26 14:47:57.62
198       199      Julia       Fawcett  2013-05-26 14:47:57.62
199       200      Thora        Temple  2013-05-26 14:47:57.62

[200 rows x 4 columns]


In [ ]:
# Insert DataFrame to Table
for index,row in df.iterrows():
    sql='''INSERT INTO actor (actor_id, first_name, last_name,last_update) VALUES (%s,%s,%s,%s)'''
    cur.execute(sql, tuple(row))
conn.commit()

In [22]:
countrydata=pd.read_csv(r'..\data\country-1677887083257.csv')
df = pd.DataFrame(countrydata)
print(df)

     country_id               country          last_update
0             1           Afghanistan  2006-02-15 09:44:00
1             2               Algeria  2006-02-15 09:44:00
2             3        American Samoa  2006-02-15 09:44:00
3             4                Angola  2006-02-15 09:44:00
4             5              Anguilla  2006-02-15 09:44:00
..          ...                   ...                  ...
104         105               Vietnam  2006-02-15 09:44:00
105         106  Virgin Islands, U.S.  2006-02-15 09:44:00
106         107                 Yemen  2006-02-15 09:44:00
107         108            Yugoslavia  2006-02-15 09:44:00
108         109                Zambia  2006-02-15 09:44:00

[109 rows x 3 columns]


In [23]:
# Insert DataFrame to Table
for index,row in df.iterrows():
    sql='''INSERT INTO country (country_id, country, last_update) VALUES (%s,%s,%s)'''
    cur.execute(sql, tuple(row))
conn.commit()

In [24]:
citydata=pd.read_csv(r'..\data\city-1677887064709.csv')
df = pd.DataFrame(citydata)
print(df)

     city_id                city  country_id          last_update
0          1  A Corua (La Corua)          87  2006-02-15 09:45:25
1          2                Abha          82  2006-02-15 09:45:25
2          3           Abu Dhabi         101  2006-02-15 09:45:25
3          4                Acua          60  2006-02-15 09:45:25
4          5               Adana          97  2006-02-15 09:45:25
..       ...                 ...         ...                  ...
595      596               Zaria          69  2006-02-15 09:45:25
596      597        Zeleznogorsk          80  2006-02-15 09:45:25
597      598         Zhezqazghan          51  2006-02-15 09:45:25
598      599            Zhoushan          23  2006-02-15 09:45:25
599      600          Ziguinchor          83  2006-02-15 09:45:25

[600 rows x 4 columns]


In [25]:
# Insert DataFrame to Table
for index,row in df.iterrows():
    sql='''INSERT INTO city (city_id, city, country_id,last_update) VALUES (%s,%s,%s,%s)'''
    cur.execute(sql, tuple(row))
conn.commit()

#### Inserting data into adress table

In [26]:
adrdata = pd.read_csv (r'..\data\address-1677887015727.csv')  
adrdata = adrdata.astype(object).where(pd.notnull(adrdata), None)
df = pd.DataFrame(adrdata)
print(df)

    address_id                   address address2      district city_id  \
0            1         47 MySakila Drive     None       Alberta     300   
1            2        28 MySQL Boulevard     None           QLD     576   
2            3         23 Workhaven Lane     None       Alberta     300   
3            4      1411 Lillydale Drive     None           QLD     576   
4            5            1913 Hanoi Way     None      Nagasaki     463   
..         ...                       ...      ...           ...     ...   
598        601       844 Bucuresti Place     None      Liaoning     242   
599        602  1101 Bucuresti Boulevard     None   West Greece     401   
600        603    1103 Quilmes Boulevard     None         Piura     503   
601        604       1331 Usak Boulevard     None          Vaud     296   
602        605      1325 Fukuyama Street     None  Heilongjiang     537   

    postal_code           phone          last_update  
0          None            None  2006-02-15 

In [27]:
# Insert DataFrame to Table

for index,row in df.iterrows():
    sql='''INSERT INTO address (address_id, address, address2 ,district,city_id, postal_code,phone,last_update) 
    VALUES (%s,%s,%s,%s,%s,%s,%s,%s)'''
    cur.execute(sql, tuple(row))
conn.commit()

address_id                       1
address          47 MySakila Drive
address2                      None
district                   Alberta
city_id                        300
postal_code                   None
phone                         None
last_update    2006-02-15 09:45:30
Name: 0, dtype: object
address_id                       2
address         28 MySQL Boulevard
address2                      None
district                       QLD
city_id                        576
postal_code                   None
phone                         None
last_update    2006-02-15 09:45:30
Name: 1, dtype: object
address_id                       3
address          23 Workhaven Lane
address2                      None
district                   Alberta
city_id                        300
postal_code                   None
phone                14033335568.0
last_update    2006-02-15 09:45:30
Name: 2, dtype: object
address_id                        4
address        1411 Lillydale Drive
address2           

address_id                     133
address          1854 Tieli Street
address2                      None
district                  Shandong
city_id                        302
postal_code                15819.0
phone               509492324775.0
last_update    2006-02-15 09:45:30
Name: 132, dtype: object
address_id                     134
address             758 Junan Lane
address2                      None
district                      Gois
city_id                        190
postal_code                82639.0
phone               935448624185.0
last_update    2006-02-15 09:45:30
Name: 133, dtype: object
address_id                          135
address        1752 So Leopoldo Parkway
address2                           None
district                  Taka-Karpatia
city_id                             345
postal_code                     14014.0
phone                    252265130067.0
last_update         2006-02-15 09:45:30
Name: 134, dtype: object
address_id                     136
address   

address_id                       234
address        1256 Bislig Boulevard
address2                        None
district                    Botosani
city_id                           86
postal_code                  50598.0
phone                 479007229460.0
last_update      2006-02-15 09:45:30
Name: 233, dtype: object
address_id                     235
address          954 Kimchon Place
address2                      None
district              West Bengali
city_id                        559
postal_code                42420.0
phone               541327526474.0
last_update    2006-02-15 09:45:30
Name: 234, dtype: object
address_id                     236
address          885 Yingkou Manor
address2                      None
district                    Kaduna
city_id                        596
postal_code                31390.0
phone               588964509072.0
last_update    2006-02-15 09:45:30
Name: 235, dtype: object
address_id                     237
address          1736 Cavite Place

address_id                              361
address        45 Aparecida de Goinia Place
address2                               None
district                     Madhya Pradesh
city_id                                 464
postal_code                          7431.0
phone                        650496654258.0
last_update             2006-02-15 09:45:30
Name: 359, dtype: object
address_id                                   362
address        482 Kowloon and New Kowloon Manor
address2                                    None
district                              Bratislava
city_id                                       90
postal_code                              97056.0
phone                             738968474939.0
last_update                  2006-02-15 09:45:30
Name: 360, dtype: object
address_id                     363
address             604 Bern Place
address2                      None
district                 Jharkhand
city_id                        429
postal_code                 537

address_id                     498
address          319 Plock Parkway
address2                      None
district                  Istanbul
city_id                        504
postal_code                26101.0
phone               854259976812.0
last_update    2006-02-15 09:45:30
Name: 496, dtype: object
address_id                                  499
address        1954 Kowloon and New Kowloon Way
address2                                   None
district                             Chimborazo
city_id                                     434
postal_code                             63667.0
phone                            898559280434.0
last_update                 2006-02-15 09:45:30
Name: 497, dtype: object
address_id                     500
address            362 Rajkot Lane
address2                      None
district                     Gansu
city_id                         47
postal_code                98030.0
phone               962020153680.0
last_update    2006-02-15 09:45:30
Name: 

address_id                     604
address        1331 Usak Boulevard
address2                      None
district                      Vaud
city_id                        296
postal_code                61960.0
phone               145308717464.0
last_update    2006-02-15 09:45:30
Name: 601, dtype: object
address_id                      605
address        1325 Fukuyama Street
address2                       None
district               Heilongjiang
city_id                         537
postal_code                 27107.0
phone                288241215394.0
last_update     2006-02-15 09:45:30
Name: 602, dtype: object


#### category-1677887044868

In [28]:
datacateg = pd.read_csv (r'..\data\category-1677887044868.csv')   
df = pd.DataFrame(datacateg)
print(df)

    category_id         name          last_update
0             1       Action  2006-02-15 09:46:27
1             2    Animation  2006-02-15 09:46:27
2             3     Children  2006-02-15 09:46:27
3             4     Classics  2006-02-15 09:46:27
4             5       Comedy  2006-02-15 09:46:27
5             6  Documentary  2006-02-15 09:46:27
6             7        Drama  2006-02-15 09:46:27
7             8       Family  2006-02-15 09:46:27
8             9      Foreign  2006-02-15 09:46:27
9            10        Games  2006-02-15 09:46:27
10           11       Horror  2006-02-15 09:46:27
11           12        Music  2006-02-15 09:46:27
12           13          New  2006-02-15 09:46:27
13           14       Sci-Fi  2006-02-15 09:46:27
14           15       Sports  2006-02-15 09:46:27
15           16       Travel  2006-02-15 09:46:27


In [29]:
# Insert DataFrame to Table
for index,row in df.iterrows():
    sql='''INSERT INTO category (category_id,name,last_update) VALUES (%s,%s,%s)'''
    cur.execute(sql, tuple(row))
conn.commit()

category_id                      1
name                        Action
last_update    2006-02-15 09:46:27
Name: 0, dtype: object
category_id                      2
name                     Animation
last_update    2006-02-15 09:46:27
Name: 1, dtype: object
category_id                      3
name                      Children
last_update    2006-02-15 09:46:27
Name: 2, dtype: object
category_id                      4
name                      Classics
last_update    2006-02-15 09:46:27
Name: 3, dtype: object
category_id                      5
name                        Comedy
last_update    2006-02-15 09:46:27
Name: 4, dtype: object
category_id                      6
name                   Documentary
last_update    2006-02-15 09:46:27
Name: 5, dtype: object
category_id                      7
name                         Drama
last_update    2006-02-15 09:46:27
Name: 6, dtype: object
category_id                      8
name                        Family
last_update    2006-02-15 09:46:27

In [37]:
langdata = pd.read_csv (r'..\data\language-1677887305001.csv')   
df = pd.DataFrame(langdata)
print(df)

   language_id                  name          last_update
0            1  English               2006-02-15 10:02:19
1            2  Italian               2006-02-15 10:02:19
2            3  Japanese              2006-02-15 10:02:19
3            4  Mandarin              2006-02-15 10:02:19
4            5  French                2006-02-15 10:02:19
5            6  German                2006-02-15 10:02:19


In [38]:
for index,row in df.iterrows():
    sql='''INSERT INTO language(language_id ,name,last_update) VALUES (%s,%s,%s)'''
    cur.execute(sql, tuple(row))
conn.commit()

In [40]:
filmdata = pd.read_csv (r'..\data\film-1677887139521.csv')   
df = pd.DataFrame(filmdata)
print(df)

     film_id              title  \
0        133    Chamber Italian   
1        384   Grosse Wonderful   
2          8    Airport Pollock   
3         98  Bright Encounters   
4          1   Academy Dinosaur   
..       ...                ...   
995      996     Young Language   
996      997         Youth Kick   
997      998       Zhivago Core   
998      999  Zoolander Fiction   
999     1000          Zorro Ark   

                                           description  release_year  \
0    A Fateful Reflection of a Moose And a Husband ...          2006   
1    A Epic Drama of a Cat And a Explorer who must ...          2006   
2    A Epic Tale of a Moose And a Girl who must Con...          2006   
3    A Fateful Yarn of a Lumberjack And a Feminist ...          2006   
4    A Epic Drama of a Feminist And a Mad Scientist...          2006   
..                                                 ...           ...   
995  A Unbelieveable Yarn of a Boat And a Database ...          2006   
996

In [41]:
for index,row in df.iterrows():
    sql='''INSERT INTO film(film_id ,title,description,release_year,language_id,rental_duration,
    rental_rate,length ,replacement_cost,rating ,last_update,special_features,full_text)
    VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'''
    cur.execute(sql, tuple(row))
conn.commit()

film_id                                                           133
title                                                 Chamber Italian
description         A Fateful Reflection of a Moose And a Husband ...
release_year                                                     2006
language_id                                                         1
rental_duration                                                     7
rental_rate                                                      4.99
length                                                            117
replacement_cost                                                14.99
rating                                                          NC-17
last_update                                   2013-05-26 14:50:58.951
special_features                                           {Trailers}
fulltext            'chamber':1 'fate':4 'husband':11 'italian':2 ...
Name: 0, dtype: object
film_id                                                           3

film_id                                                           125
title                                                 Cassidy Wyoming
description         A Intrepid Drama of a Frisbee And a Hunter who...
release_year                                                     2006
language_id                                                         1
rental_duration                                                     5
rental_rate                                                      2.99
length                                                             61
replacement_cost                                                19.99
rating                                                          NC-17
last_update                                   2013-05-26 14:50:58.951
special_features                   {Commentaries,"Behind the Scenes"}
fulltext            'agent':17 'cassidi':1 'drama':5 'frisbe':8 'h...
Name: 129, dtype: object
film_id                                                          

film_id                                                           218
title                                               Deceiver Betrayed
description         A Taut Story of a Moose And a Squirrel who mus...
release_year                                                     2006
language_id                                                         1
rental_duration                                                     7
rental_rate                                                      0.99
length                                                            122
replacement_cost                                                22.99
rating                                                          NC-17
last_update                                   2013-05-26 14:50:58.951
special_features    {Trailers,Commentaries,"Deleted Scenes","Behin...
fulltext            'ancient':18 'betray':2 'build':14 'deceiv':1 ...
Name: 219, dtype: object
film_id                                                          

film_id                                                           320
title                                           Flamingos Connecticut
description         A Fast-Paced Reflection of a Composer And a Co...
release_year                                                     2006
language_id                                                         1
rental_duration                                                     4
rental_rate                                                      4.99
length                                                             80
replacement_cost                                                28.99
rating                                                          PG-13
last_update                                   2013-05-26 14:50:58.951
special_features                                           {Trailers}
fulltext            'cat':18 'compos':10,13 'connecticut':2 'deser...
Name: 320, dtype: object
film_id                                                          

film_id                                                           414
title                                             Hellfighters Sierra
description         A Taut Reflection of a A Shark And a Dentist w...
release_year                                                     2006
language_id                                                         1
rental_duration                                                     3
rental_rate                                                      2.99
length                                                             75
replacement_cost                                                23.99
rating                                                             PG
last_update                                   2013-05-26 14:50:58.951
special_features               {"Deleted Scenes","Behind the Scenes"}
fulltext            'battl':15 'boat':17 'dentist':12 'georgia':20...
Name: 414, dtype: object
film_id                                                          

film_id                                                           517
title                                                Lesson Cleopatra
description         A Emotional Display of a Man And a Explorer wh...
release_year                                                     2006
language_id                                                         1
rental_duration                                                     3
rental_rate                                                      0.99
length                                                            167
replacement_cost                                                28.99
rating                                                          NC-17
last_update                                   2013-05-26 14:50:58.951
special_features               {"Deleted Scenes","Behind the Scenes"}
fulltext            'boy':16 'build':14 'cleopatra':2 'display':5 ...
Name: 517, dtype: object
film_id                                                          

film_id                                                           618
title                                               Necklace Outbreak
description         A Astounding Epistle of a Database Administrat...
release_year                                                     2006
language_id                                                         1
rental_duration                                                     3
rental_rate                                                      0.99
length                                                            132
replacement_cost                                                21.99
rating                                                             PG
last_update                                   2013-05-26 14:50:58.951
special_features          {Trailers,Commentaries,"Behind the Scenes"}
fulltext            'administr':9 'astound':4 'california':20 'cat...
Name: 618, dtype: object
film_id                                                          

film_id                                                           719
title                                                   Records Zorro
description         A Amazing Drama of a Mad Scientist And a Compo...
release_year                                                     2006
language_id                                                         1
rental_duration                                                     7
rental_rate                                                      4.99
length                                                            182
replacement_cost                                                11.99
rating                                                             PG
last_update                                   2013-05-26 14:50:58.951
special_features                                {"Behind the Scenes"}
fulltext            'amaz':4 'build':15 'compos':12 'drama':5 'hus...
Name: 719, dtype: object
film_id                                                          

film_id                                                           829
title                                                    Spinal Rocky
description         A Lacklusture Epistle of a Sumo Wrestler And a...
release_year                                                     2006
language_id                                                         1
rental_duration                                                     7
rental_rate                                                      2.99
length                                                            138
replacement_cost                                                12.99
rating                                                          PG-13
last_update                                   2013-05-26 14:50:58.951
special_features                      {Commentaries,"Deleted Scenes"}
fulltext            'california':19 'defeat':15 'epistl':5 'explor...
Name: 828, dtype: object
film_id                                                          

film_id                                                           926
title                                            Untouchables Sunrise
description         A Amazing Documentary of a Woman And a Astrona...
release_year                                                     2006
language_id                                                         1
rental_duration                                                     5
rental_rate                                                      2.99
length                                                            120
replacement_cost                                                11.99
rating                                                          NC-17
last_update                                   2013-05-26 14:50:58.951
special_features                          {Trailers,"Deleted Scenes"}
fulltext            'abandon':19 'amaz':4 'astronaut':11 'document...
Name: 925, dtype: object
film_id                                                          

In [45]:
storedata = pd.read_csv (r'..\data\store-1677887380221.csv')   
df = pd.DataFrame(storedata)
print(df)

   store_id  manager_staff_id  address_id          last_update
0         1                 1           1  2006-02-15 09:57:12
1         2                 2           2  2006-02-15 09:57:12


In [47]:
for index,row in df.iterrows():
    sql='''INSERT INTO customer(store_id ,address_id,last_update)
    VALUES (%s,%s,%s,%s)'''
    cur.execute(sql, tuple(row))
conn.commit()

DataError: 1136 (21S01): Column count doesn't match value count at row 1

In [ ]:
custdata = pd.read_csv (r'..\data\customer-1677887114572.csv')   
df = pd.DataFrame(custdata)
print(df)

In [44]:
for index,row in df.iterrows():
    sql='''INSERT INTO customer(customer_id ,store_id,first_name,last_name,email,address_id,
    activebool, create_date ,last_update,active)
    VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'''
    cur.execute(sql, tuple(row))
conn.commit()

IntegrityError: 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`dvdrentaldb`.`customer`, CONSTRAINT `customer_ibfk_1` FOREIGN KEY (`store_id`) REFERENCES `store` (`store_id`))

     customer_id  store_id first_name  last_name  \
0            524         1      Jared        Ely   
1              1         1       Mary      Smith   
2              2         1   Patricia    Johnson   
3              3         1      Linda   Williams   
4              4         2    Barbara      Jones   
..           ...       ...        ...        ...   
594          595         1   Terrence  Gunderson   
595          596         1    Enrique   Forsythe   
596          597         1    Freddie     Duggan   
597          598         1       Wade   Delvalle   
598          599         2     Austin    Cintron   

                                     email  address_id  activebool  \
0             jared.ely@sakilacustomer.org         530        True   
1            mary.smith@sakilacustomer.org           5        True   
2      patricia.johnson@sakilacustomer.org           6        True   
3        linda.williams@sakilacustomer.org           7        True   
4         barbara.jones@s